## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

def calibration():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            cv2.imshow('img',img)
            cv2.waitKey(500)

    cv2.destroyAllWindows()

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    
    return ret, mtx, dist, rvecs, tvecs

In [107]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 255
    binary_output = sxbinary
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobel = np.sqrt(np.square(sobelx) + np.square(sobely))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 255
    binary_output = sxbinary
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    scaled_sobel = np.arctan2(abs_sobely, abs_sobelx)
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 255
    binary_output = sxbinary
    return binary_output

In [99]:
def hls_select(img, channel = "S", thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    if channel == "H":
        color = hls[:,:,0]
    elif channel == "L":
        color = hls[:,:,1]
    else:
        color = hls[:,:,2]
    # 3) Return a binary image of threshold result
    binary = np.zeros_like(color)
    binary[(color > thresh[0]) & (color <= thresh[1])] = 255
    binary_output = binary # placeholder line
    return binary_output

In [109]:
original = cv2.imread('test_images/test2.jpg')
#Undistort Image
undist = cv2.undistort(original, mtx, dist, None, mtx)

In [154]:
hls_img = hls_select(undist, thresh=(100,255))

cv2.imshow('hls_img', hls_img)
cv2.imshow('original', original)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [146]:
# img = abs_sobel_thresh(undist, orient = 'x', thresh_min = 20, thresh_max = 100)
mag_img = mag_thresh(undist, mag_thresh=(50, 255))
dir_img = dir_threshold(undist, thresh=(np.pi/4,2*np.pi/4))
sobelx_img = abs_sobel_thresh(undist, orient='x', thresh_min=50, thresh_max=100)
sobely_img = abs_sobel_thresh(undist, orient='y', thresh_min=50, thresh_max=100)

cv2.imshow('sobelx', sobelx_img)
cv2.imshow('sobely', sobely_img)
# cv2.imshow('dir_img', dir_img)
cv2.imshow('mag_img', mag_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [156]:
combined = np.zeros_like(dir_img)
combined[((sobelx == 255) & (sobely == 255)) | ((mag_img == 255) & (dir_img == 255)) | (hls_img == 255)] = 255
cv2.imwrite('output_images/combinethreshImage.jpg', combined)
cv2.imshow("combine", combined)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
ret, mtx, dist, rvecs, tvecs = calibration()

In [2]:
#Calculating Distortion Matrix
ret, mtx, dist, rvecs, tvecs = calibration()
original = cv2.imread('camera_cal/calibration1.jpg')
undist = cv2.undistort(original, mtx, dist, None, mtx)
cv2.imwrite('output_images/undistortImage.jpg', undist)
# cv2.imshow('original', original)
# cv2.imshow('undist', undist)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True